### Database works.
In this **notebook** we have a lot of good stuff:

In [2]:
%load_ext sql
import os, getpass, time, xuleta

user = input("Username: ")
password = getpass.getpass('Enter your password: ')
database = input("Database: ")
connection_string = "postgresql://%s:%s@localhost/%s" %(user,password,database)
%sql $connection_string

Username: denis
Enter your password: ········
Database: drought


'Connected: denis@drought'

### INPUT PARAMETERS
Make sure the **table** matches the list provide in the NODATA DICTIONARY block

In [18]:
schema = "GLEAM"
table = "swdi_surface"
imfolder = "/home/denis/Downloads/swdi_surface/"

t0 = time.time()
if imfolder[-1] != "/":
    imfolder = imfolder + "/"

nodatas = {'pet':0,'ple':0,'le':0,'et':0,'gpp':-32767,'gpp_f2':1,'albedo':32767,
          'psn':32767,'ndvi':1,'lstd':65535,'lstn':65535,'lst_day':0,'lst_night':0,
          'vpd':-32768,'lai':'nan','fpar':'nan','zscore_dec':-32768,'anom_dec':-32768,'t04':-9999,'t12':-9999,
          'prec':-9999,'landcover':0,'ndvi_f2':-30000,'SM_root_abs':'nan','SM_root_anom':'nan',
           'SM_surf_abs':'nan','SM_surf_anom':'nan','swdi_root':'nan','swdi_surface':'nan'}
nodata = str(nodatas[table])
print("nodata value for %s is set to %s" %(table.upper(),nodata))

nodata value for SWDI_SURFACE is set to nan


In [20]:
xuleta.renamedate(imfolder,oldf='%Y_%j',newf='%Y-%m-%d')

Done!


### Creating Schema and table

It might not be necessary to create a schema.

In [21]:
SQL = ("CREATE SCHEMA %s;" %(schema))
%sql $SQL

(psycopg2.ProgrammingError) schema "gleam" already exists
 [SQL: 'CREATE SCHEMA GLEAM;']


In [22]:
#CREATING TABLE
SQL = ("""
CREATE TABLE %s.%s(
rid serial NOT NULL,
rast RASTER,
filename text,
acquisition date,
CONSTRAINT %s_%s_pkey
PRIMARY KEY (rid));
CREATE INDEX %s_%s_wkb_rast_idx
ON %s.%s
USING GIST (ST_ConvexHull(rast)); """ 
       %(schema,table,schema,table,schema,table,schema,table))
%sql $SQL

Done.
Done.


[]

In [23]:
%%time
comment = table.upper() + " data for " + schema + "."
SQL = ("""
COMMENT ON TABLE %s.%s 
IS '%s'
"""
    %(schema,table,comment))
%sql $SQL

Done.
CPU times: user 1.71 ms, sys: 456 µs, total: 2.17 ms
Wall time: 31.7 ms


### Importing images to the database

You can watch the verbose in the terminal to make sure that things are going ok.

In [24]:
importraster = "raster2pgsql -a -C -F -M -s 4326 -t 100x100 -N %s %s*.tif %s.%s | psql -p 5432 -d drought -U denis" %(nodata,imfolder,schema,table)
print(importraster)

raster2pgsql -a -C -F -M -s 4326 -t 100x100 -N nan /home/denis/Downloads/swdi_surface/*.tif GLEAM.swdi_surface | psql -p 5432 -d drought -U denis


In [25]:
%%time
os.system(importraster)

CPU times: user 402 µs, sys: 4.25 ms, total: 4.66 ms
Wall time: 2.66 s


0

#### Creating date index from filename

In [26]:
%%time
SQL = ("""
UPDATE %s.%s
    SET acquisition = date(
    substring(filename FROM 1 FOR 4) || '-' ||
    substring(filename FROM 6 FOR 2) || '-' ||
    substring(filename FROM 9 FOR 2))::date;
    
CREATE INDEX %s_acquisition_idx
    ON %s.%s (acquisition);
"""
      %(schema,table,table,schema,table))
#execute query
%sql $SQL

780 rows affected.
Done.
CPU times: user 2.42 ms, sys: 3.69 ms, total: 6.11 ms
Wall time: 543 ms


In [12]:
taime = (time.time() - t0)/60
xuleta.shemale(taime)

To:denismeia@icloud.com
From: denismeia@gmail.com
Subject:PROCESS DONE 

done!


### Testing

In [13]:
point = [str(-92.891),str(42.436)]#US
point = [str(-38.8),str(-8.03)] #PB
point = [str(-52.84),str(-24.48)] #PR

dates = [('1980-10-29'),('2017-01-09')]

In [14]:
# POINT BASED TEST
SQL = ("""SELECT 
        rid, 
        acquisition,
        ST_Value(rast, ST_SetSRID(ST_MakePoint(%s,%s), 4326)) AS %s
    FROM %s.%s
         WHERE ST_Intersects(ST_SetSRID(ST_MakePoint(%s,%s), 4326), rast)
           AND acquisition BETWEEN '%s' AND '%s'
    ORDER BY acquisition;
"""
%(point[0],point[1],table,schema,table,point[0],point[1],dates[0],dates[1]))
print(SQL)
%sql $SQL

SELECT 
        rid, 
        acquisition,
        ST_Value(rast, ST_SetSRID(ST_MakePoint(-52.84,-24.48), 4326)) AS swdi_root
    FROM GLEAM.swdi_root
         WHERE ST_Intersects(ST_SetSRID(ST_MakePoint(-52.84,-24.48), 4326), rast)
           AND acquisition BETWEEN '1980-10-29' AND '2017-01-09'
    ORDER BY acquisition;

730 rows affected.


rid,acquisition,swdi_root
1,2003-01-01,-2.01476311683655
2,2003-01-08,-2.10800313949585
3,2003-01-15,-2.32551860809326
4,2003-01-22,-0.0472123995423317
5,2003-01-29,-1.03352200984955
6,2003-02-05,-1.0842912197113
7,2003-02-12,-0.2883320748806
8,2003-02-19,0.452064484357834
9,2003-02-26,-0.271842539310455
10,2003-03-05,0.0602447204291821


### extract data

In [15]:
%%time
SQL = ("""
SELECT
    geocodig_m , sigla, nome_munic, acquisition,
    (St_SummaryStats(St_Union(ST_Clip(rast,1,geom, true)))).*
INTO consultas_br.ndvi_f2_big_sul_crop
FROM modis_sul_250.ndvi_f2 , public.big_sul_crop
WHERE st_intersects(rast,geom)
GROUP BY geocodig_m , sigla, nome_munic, acquisition
ORDER BY acquisition;
COMMENT ON TABLE consultas_br.ndvi_f2_big_sul_crop
IS 'MOD NDVI for part of RS, SC, PR and part of SP';
""")
%sql $SQL

(psycopg2.ProgrammingError) relation "modis_sul_250.ndvi_f2" does not exist
LINE 5: FROM modis_sul_250.ndvi_f2 , public.big_sul_crop
             ^
 [SQL: 'SELECT\n    geocodig_m , sigla, nome_munic, acquisition,\n    (St_SummaryStats(St_Union(ST_Clip(rast,1,geom, true)))).*\nINTO consultas_br.ndvi_f2_big_sul_crop\nFROM modis_sul_250.ndvi_f2 , public.big_sul_crop\nWHERE st_intersects(rast,geom)\nGROUP BY geocodig_m , sigla, nome_munic, acquisition\nORDER BY acquisition;']
CPU times: user 3.23 ms, sys: 0 ns, total: 3.23 ms
Wall time: 2.6 ms


** Polygon based **

In [16]:
dates = [('2007-10-29'),('2008-01-09')]

In [17]:
#POLYGON BASED TEST - BRASIL
SQL = ("""SELECT 
    geocodig_m , sigla, nome_munic, acquisition,
    (St_SummaryStats(St_Union(ST_Clip(rast,1,geom, true)))).*
FROM %s.%s , public.big_sul_crop
WHERE st_intersects(rast,geom)
    AND acquisition BETWEEN '%s' AND '%s'
    AND sigla IN ('PR')
GROUP BY geocodig_m , sigla, nome_munic, acquisition
ORDER BY acquisition;
"""
       %(schema,table,dates[0],dates[1]))
print(SQL)
%sql $SQL

SELECT 
    geocodig_m , sigla, nome_munic, acquisition,
    (St_SummaryStats(St_Union(ST_Clip(rast,1,geom, true)))).*
FROM GLEAM.swdi_root , public.big_sul_crop
WHERE st_intersects(rast,geom)
    AND acquisition BETWEEN '2007-10-29' AND '2008-01-09'
    AND sigla IN ('PR')
GROUP BY geocodig_m , sigla, nome_munic, acquisition
ORDER BY acquisition;

3135 rows affected.


geocodig_m,sigla,nome_munic,acquisition,count,sum,mean,stddev,min,max
4100103.0,PR,Abati,2007-10-29,0,None,None,None,None,None
4100707.0,PR,Alto Piquiri,2007-10-29,0,None,None,None,None,None
4100806.0,PR,Alvorada do Sul,2007-10-29,1,-9.06481456756592,-9.06481456756592,0.0,-9.06481456756592,-9.06481456756592
4101002.0,PR,Ampre,2007-10-29,0,None,None,None,None,None
4101051.0,PR,Anahy,2007-10-29,0,None,None,None,None,None
4101101.0,PR,Andir,2007-10-29,1,-8.11718654632568,-8.11718654632568,0.0,-8.11718654632568,-8.11718654632568
4101150.0,PR,ngulo,2007-10-29,0,None,None,None,None,None
4101408.0,PR,Apucarana,2007-10-29,0,None,None,None,None,None
4101507.0,PR,Arapongas,2007-10-29,0,None,None,None,None,None
4101606.0,PR,Arapoti,2007-10-29,0,None,None,None,None,None


In [ ]:
result = %sql $SQL
result = result.DataFrame()

In [ ]:
result.sigla.unique()

In [ ]:
result[(result.sigla == 'PB') & (result['count'] == 0)]

### Getting results into tables

In [14]:
# the st_buffer(geom,0) prevents some topology errors
SQL = ("""
SELECT
    geocodig_m , sigla, nome_munic, acquisition,
    (St_SummaryStats(St_Union(ST_Clip(rast,1,st_buffer(geom, 0), true)))).*
INTO consultas_br.sul_chirps_zscore
FROM chirps.zscore_dec , public.big_sul
WHERE st_intersects(rast,geom)
--    AND acquisition BETWEEN '2007-10-29' AND '2007-11-19'
    AND sigla IN ('PR','SC','RS')
GROUP BY geocodig_m , sigla, nome_munic, acquisition
ORDER BY acquisition;
""")
%sql $SQL

734184 rows affected.


[]

In [15]:
xuleta.shemale()

To:denismeia@icloud.com
From: denismeia@gmail.com
Subject:PROCESS DONE 

done!


In [10]:
dates = [('2002-07-01'),('2018-03-09')]
l1 = ["modis_lai","modis_et"]
l2 = ["lai","et"]
for i,j in zip(l1,l2):
    SQL = ("""SELECT 
    geocodig_m , sigla, nome_munic, acquisition,
    (St_SummaryStats(St_Union(ST_Clip(rast,1,geom, true)))).*
    INTO consultas_br.%s
    FROM %s.%s , public.big_sul_crop
    WHERE st_intersects(rast,geom)
    AND acquisition BETWEEN '%s' AND '%s'
    AND sigla IN ('PR','RS','SC')
    GROUP BY geocodig_m , sigla, nome_munic, acquisition
    ORDER BY acquisition;
    """
       %(j,i,j,dates[0],dates[1]))
    print(SQL)
    %sql $SQL

SELECT 
    geocodig_m , sigla, nome_munic, acquisition,
    (St_SummaryStats(St_Union(ST_Clip(rast,1,geom, true)))).*
    INTO consultas_br.lai
    FROM modis_lai.lai , public.big_sul_crop
    WHERE st_intersects(rast,geom)
    AND acquisition BETWEEN '2002-07-01' AND '2018-03-09'
    AND sigla IN ('PR','RS','SC')
    GROUP BY geocodig_m , sigla, nome_munic, acquisition
    ORDER BY acquisition;
    
403650 rows affected.
SELECT 
    geocodig_m , sigla, nome_munic, acquisition,
    (St_SummaryStats(St_Union(ST_Clip(rast,1,geom, true)))).*
    INTO consultas_br.et
    FROM modis_et.et , public.big_sul_crop
    WHERE st_intersects(rast,geom)
    AND acquisition BETWEEN '2002-07-01' AND '2018-03-09'
    AND sigla IN ('PR','RS','SC')
    GROUP BY geocodig_m , sigla, nome_munic, acquisition
    ORDER BY acquisition;
    
378495 rows affected.


In [11]:
xu.shemale()

NameError: name 'xu' is not defined